In [ ]:
import itertools
import numpy as np
import tensorflow as tf
from tensorflow import keras
import sys
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from joblib import dump, load
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score

In [ ]:
# load data
X_mfccs = np.load('../util/X_mfccs/X_mfccs_4000.npy')
X_transcripts = np.load('../util/X_transcripts/X_transcripts_4000.npy')
Y = np.load('../util/labels/labels_4000.npy')
X_train_mfcc, X_test_mfcc, X_train_transcript, X_test_transcript, y_train, y_test = train_test_split(X_mfccs,X_transcripts,Y,test_size=0.1, random_state = 42)

# load models
rf = load('rf.joblib')
dl = tf.keras.models.load_model("tuned_model")

In [ ]:
# predict using rf
rf_predictions = rf.predict_proba(X_test_mfcc.reshape(X_test_mfcc.shape[0], -1))[:,1]

# predict using dl
dl_predictions = dl.predict([X_test_mfcc,X_test_transcript]).ravel()

In [ ]:
# plot roc curves
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, dl_predictions)
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, rf_predictions)

auc_keras = auc(fpr_keras, tpr_keras)
auc_rf = auc(fpr_rf, tpr_rf)

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='DL (area = {:.3f})'.format(auc_keras))
plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

In [ ]:
# print AP scores

average_precision_rf = average_precision_score(y_test, rf_predictions)

print('Average random forest precision-recall score: {0:0.2f}'.format(
      average_precision_rf))

average_precision_dl = average_precision_score(y_test, dl_predictions)

print('Average deep learning precision-recall score: {0:0.2f}'.format(
      average_precision_dl))

In [ ]:
# plot decision matrix for random forest

disp = plot_confusion_matrix(rf, X_test_mfcc.reshape(X_test_mfcc.shape[0], -1), y_test,
                                 cmap=plt.cm.Blues)
disp.ax_.set_title("Random Forest")


In [ ]:
# we need to define our own function to plot the decision matrix for deep learning, 
# since it's not a scikit-learn classifier

def plot_confusion_matrix_custom(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# plot confusion matrix for deep learning

cm = confusion_matrix(y_test, np.round(dl_predictions))
plot_confusion_matrix_custom(cm, ["0", "1"], "Deep Learning")

In [ ]:
# custom functions to print accuracies

def find_counts(predictions, truth):
    TP = 0; TN = 0; FP = 0; FN = 0
    for i in range(predictions.shape[0]):
        if predictions[i] < 0.5 and truth[i] == 0:
            TN += 1
        elif predictions[i] >= 0.5 and truth[i] == 1:
            TP += 1
        elif predictions[i] < 0.5 and truth[i] == 1:
            FN += 1
        elif predictions[i] >= 0.5 and truth[i] == 0:
            FP += 1
    return TP, TN, FP, FN

def find_accuracies(TP, TN, FP, FN):
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    a_0 = TP / (TP + FN)
    a_1 = TN / (TN + FP)
    balanced_accuracy = 1/2 * (a_0 + a_1)
    return accuracy, balanced_accuracy, a_0, a_1

In [ ]:
# print accuracies 

TP, TN, FP, FN = find_counts(rf_predictions, y_test)
print(find_accuracies(TP, TN, FP, FN))